# Step 1: Assigning issuer names to securities in a portfolio

Source: Eagle 'Reg 28 Report - Incl Effective Exposure' with nine columns: 0. 'Entity Name', 1. 'Investment Type', 2. 'i Issue Name', 3. 'Primary Asset ID', 4. 'CCY', 5. 'Reg28 Classification', 6. 'End Market Value', 7. 'Percentage', 8. 'Closing Exposure PA'

#### Resources

input file                       P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx

structures                       P:\Working Folders\Hilton\W\structures.xlsm

Reg28 sheet format and formula   P:\Working Folders\Hilton\W\Reg28 Checker Format and Formulae.docx

issuer results file              P:\Working Folders\Hilton\W\issuers_1_done.xlsx

merged file                      P:\Working Folders\Hilton\W\merged.xlsx

output file                      P:\Working Folders\Hilton\W\output.xlsx

#### More resources
 
 'else' in a list comprehension<br>
 https://stackoverflow.com/questions/2951701/is-it-possible-to-use-else-in-a-list-comprehension
 
convert string to variable<br>
https://www.codeproject.com/Questions/1222606/Python-convert-string-and-variable-to-list-name

apply function to each element of a list<br>
https://stackoverflow.com/questions/25082410/apply-function-to-each-element-of-a-list

replace Vlookup in Python using Pandas<br>
https://www.youtube.com/watch?v=AHS925L8JVk&t=9s

In [1]:
# create a time difference function
import datetime

def timediff(start, end, decimals = 1):
    if   int((end - start) / 3600) > 0: # non-zero hours
        return str(int(  (end - start) / 3600))           + 'hr '  + \
               str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    elif int((end - start) /   60) > 0: # non-zero hours and minutes
        return str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    else:
        return str(round((end - start) %   60, decimals)) + 'sec'

In [2]:
# utility function to open an excel file, .xls or .xlsx
def open_xl_file(file_name_and_path):
    import win32com.client as win32 # library to convert xls to xlsx
    excel               = win32.gencache.EnsureDispatch('Excel.Application')
    excel.DisplayAlerts = False # suppress the warning dialogue
    excel.Workbooks.Open(file_name_and_path)
    excel.DisplayAlerts = True # unsuppress the warning dialogue

In [3]:
# libraries, libraries!

    # time, to determine run durations
import time
start_time          = time.time()
start_time_overlord = time.time()

    # general libraries
import pandas   as     pd        # for dataframes
import numpy    as     np        # for np.NaN
import re                        # for regex
from   re       import search    # for regex
from   datetime import datetime  # for script run durations
import os, os.path               # for BESA folder contents


    # utilise openpyxl tools to add excel features to results sheet
import openpyxl        as     px                                          # for adding sort filters to the excel sheet
from   openpyxl.cell   import Cell                                        # to format cells
from   openpyxl.styles import Alignment, Color, PatternFill, Font, Border # to format cells

    # access Excel application to open results sheet
import  win32com.client as win32                                          # library to convert xls to xlsx
excel = win32.gencache.EnsureDispatch('Excel.Application')                # to open excel application

print(timediff(start_time, time.time()))

# Possible error message when loading libraries, labraries!:
  # TypeError: This COM object can not automate the makepy process - please run makepy manually for this object
  # Solution: close all running Excel applications
  # https://stackoverflow.com/questions/57030278/typeerror-this-com-object-can-not-automate-the-makepy-process-please-run-make


0.7sec


In [4]:
# set paths
start_time = time.time()

pthCompliance    = r'P:\Investment Operations\GRC\Compliance'       # path to PIM compliance folder
pthW             = r'P:\Working Folders\Hilton\W'                   # path to working folder
pthBondData      = r'P:\Investment Operations\GRC\Compliance\Reporting Requirements\Monthly Reports\MonthEnd17.xlsm' #infra
input_filename   = r'\Reg 28 Report - input.xlsx'                   # input file name
results_filename = r'\issuers_1_done.xlsx'                          # output file name
#pthBESAFiles    = r'\\fileserver\pim-documents\AAPIM DEAL DOWNLOAD\ADMIN SETTLEMENT FILES\BESA MTM Files' # set folder path
pthBESAFiles_new = r'P:\Shared Folders\Data\MTM' # set folder path for fixed interest securities market data

# calendar for BESA files
cal = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,'Nov': 11, 'Dec': 12}

print(timediff(start_time, time.time()))

0.0sec


In [5]:
# get filename for latest BESA file

fld = [s for s in os.listdir(pthBESAFiles) if s.isdigit()]
yr  = max(fld) # get name of newest year folder
z   = os.listdir(os.path.join(pthBESAFiles, yr)) # path including latest year
mth = [cal[key] for s in z for key in cal if key in s] # list of month numbers
fls = os.listdir(os.path.join(pthBESAFiles, max(fld), z[mth.index(max(mth))])) # list of files in year sub-folder
ymd = [filenm.strip('MTMDetailed').strip('.xls') for filenm in fls] # list of BESA xls file dates
pth = os.path.join(pthBESAFiles, max(fld), z[mth.index(max(mth))]) # path name including latest year
bsa = pth + r'\MTMDetailed' + max(ymd) + '.xls'

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\\\fileserver\\pim-documents\\AAPIM DEAL DOWNLOAD\\ADMIN SETTLEMENT FILES\\BESA MTM Files'

In [ ]:
# utility function to check if a string contains a list element

def res(t_list, t_string): # https://www.geeksforgeeks.org/python-test-if-string-contains-element-from-list/
    return bool([ele for ele in t_list if(ele in t_string)])

In [ ]:
# (1) read in a Reg 28 file from Eagle as a dataframe called 'example' to use as input
start_time = time.time()

example0 = pd.read_excel(pthW + input_filename, usecols = 'A:I').dropna(subset = ['Entity Name'])

print(timediff(start_time, time.time()))

In [ ]:
# (2) get report date - https://stackoverflow.com/questions/43544514/pandas-read-specific-excel-cell-value-into-a-variable
k = pd.read_excel(pthW + input_filename, index_col = None, usecols = 'K', nrows = 0)
                            
report_date = list(k)[0] # https://realpython.com/lessons/list-methods-return-values/

print( ' Report date   : ' + str(report_date) + '\n', \
        'BESA file date: ' + max(ymd))

In [ ]:
# (3) remove rows with zero Rand values for 'Closing Exposure PA' and zero Rand values for 'End Market Value'
#example = example0[(example0['Closing Exposure PA'] != 0) | (example0['End Market Value'] != 0)]
example = example0[(example0['Closing Exposure PA'] != 0)]  
#example = example0
print(   f'Number of zero effective exposure rows removed: {len(example0) - len(example)}',
       f'\nFinal number of rows                          : {len(example)}')

In [ ]:
# get regex structures as dataframes
start_time   = time.time()

structures   = pthW          + r'\structures.xlsm'
pthCLNs      = pthCompliance + r'\MCaps\CLNs.xlsx'
pthMedCirc   = pthCompliance + r'\Medical Schemes\20220117 MSA Circ6 Categorisations.xlsx'
pthMedCirc12 = pthCompliance + r'\Medical Schemes\20230317 MSA Circ12 Categorisations.xlsx'
pthSttlmnt   = pthCompliance + r'\Daily\2A - Fund Codes, Breach Register.xlsx'

dfrgx        = pd.read_excel(structures,   sheet_name = 'dates',                   usecols = ['date_regex', 'format', 'alternative format'])                                           # import the regex date formats
sa_hols      = pd.read_excel(structures,   sheet_name = 'hols',                    usecols = ['sa_hols'])  # import South African holidays
clns         = pd.read_excel(pthCLNs,      sheet_name = 'CLN',                     usecols = ['Code', 'Counterparty Long Name', 'Issuer Long Name'])
med_circ6    = pd.read_excel(pthMedCirc,   sheet_name = 'JSEDebt&Equity31Dec2021', usecols = ['Alpha', '2022Circ6'])          # prior med scheme circular
med_circ12   = pd.read_excel(pthMedCirc12, sheet_name = 'JSEDebt&Equity31Dec2022', usecols = ['Alpha', '2023Circ12'])         # latest med scheme circular
sttlmnt      = pd.read_excel(pthSttlmnt,   sheet_name = 'Sttlmnt',                 usecols = ['Fund', 'Custodian', 'SAFEX'])  # fund settlement accounts
bond_data    = pd.read_excel(pthBondData,  sheet_name = 'Bonds',                   usecols = ['Bond Code', 'Guarantee Type']) # bond data
besa_data    = pd.read_excel(bsa,          sheet_name = 'MTM',     skiprows = 5,   usecols = ['Bond Code']) # BESA data
accr         = pd.read_excel(structures,   sheet_name = 'accr',                    usecols = ['accruals']) #https://datatofish.com/pandas-dataframe-to-series/
marg         = pd.read_excel(structures,   sheet_name = 'accr',                    usecols = ['margins'])  #https://datatofish.com/pandas-dataframe-to-series/
issrgx       = pd.read_excel(structures,   sheet_name = 'issuers',                 usecols = ['description', 'id', 'name']) # 1st 3 columns
rgxid        = issrgx['id'].dropna()
rgxdesc      = issrgx['description'].dropna() # https://stackoverflow.com/questions/46218652/python-pandas-unique-value-ignoring-nan

accr_list    = [x for e in accr.values.tolist() for x in e] #https://bobbyhadz.com/blog/python-remove-square-brackets-from-list
margin_list  = [x for e in marg.dropna().values.tolist() for x in e]

print(timediff(start_time, time.time()))

In [ ]:
# function to derive ISSUER from INSTRUMENT DESCRIPTION ("i Issue Name" column)
def issuer_desc(txt):
    for pattern in rgxdesc:
        if re.search(pattern, str(txt).upper()):
            return issrgx.loc[issrgx['description'] == pattern].iat[0,2] # note 'break' within the for loop   
            break
        
txt1 = 'The quick brown fox jumped mercedes over the lazy dog.'
txt2 = 'IVTP'
print(issuer_desc(txt1), issuer_desc(txt2)) # test if a variable is a string

# https://pythonprinciples.com/blog/check-if-var-is-string/ 

In [ ]:
# function to derive ISSUER from INSTRUMENT ID ('Primary Asset ID' column)
def issuer_id(txt):
    for pattern in rgxid:
        if re.search(pattern, str(txt).upper()): # check if the pattern exists in the txt
            return issrgx.loc[issrgx['id'] == pattern].iat[0,2] # note the 'break' within the for loop 
            break

txt2 = 'SBT054 the lazy dog'
print(issuer_id(txt2))

In [ ]:
# concat 'descr' and 'id' columns - https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-pandas-dataframe
issrgx['desc+id'] = issrgx[['description', 'id']].astype(str).agg('|'.join, axis=1)
issrgx.drop(['id', 'description'], axis = 1, inplace = True)

In [ ]:
# function to derive issuer from instrument description and instrument id
def issuer_did(txt):
        for pattern in issrgx['desc+id']:
            if re.search(pattern, str(txt).upper()):
                return issrgx.loc[issrgx['desc+id'] == pattern].iat[0,0] # note 'break' within the for loop   
                break
        
txt1 = 'The quick brown fox jumped mercedes over the lazy dog.'
txt2 = 'SBT054 the lazy dog'
txt3 = 'R2023 7.75% 280223'
txt4 = 'TDBAR20230404'
txt5 = '18SEP23 USDZAR 17P'
print(issuer_did(txt1), issuer_did(txt2), issuer_did(txt3), issuer_did(txt4), issuer_did(txt5)) # test if a variable is a string
# https://pythonprinciples.com/blog/check-if-var-is-string/ 

In [ ]:
# function to derive margin account
def marg(txt):
    pattern = '(?!\d\d)MARG(?!\d\d)'                 #https://www.regular-expressions.info/lookaround.html
    if re.search(pattern, str(txt).upper()):
        return 'm'
        
txt1 = 'The quick brown fox jumped BARCMARGUSD over the lazy dog.'
txt2 = 'MAR23 ALSI.'
txt3 = 'QHFD2RCSU3QV0MAR'
print(marg(txt1), marg(txt2), marg(txt3))

In [ ]:
# function that extracts date from string
def datex(txt):
    try:
        for pattern in dfrgx['date_regex']:
            if re.search(pattern, txt.title()):
                break
        return datetime.strptime(re.search(pattern, txt.title()).group(),dfrgx.loc[dfrgx['date_regex'] == pattern].iat[0,1]).strftime("%d%b%Y")

    # manage exceptions:
    except ValueError as ve:
        #print(f'ValueError {ve}')
        return None
    except TypeError as te:
        #print(f'TypeError {te}')
        return None
    except AttributeError as ae:
        #print(f'AttributeError {ae}')
        return None
        
#test the datex function
txt1  = 'BNP Paribas Personal Finance SA Ltd FRN BPPF31 Jb3+95 17-Aug-13'
txt2  = 'Nedbank Semi-Annual Step-Up Note FRN JB3+115 110723  (20240711)'
txt3  = 'erf'

print(datex(txt1), type(datex(txt1)), datex(txt2), type(datex(txt2)),datex(txt3), type(datex(txt3)))

In [ ]:
# function to assign med scheme category based on Circular 6 of 2022 from the CMS
def medcirc062022(txt):
    #if med_circ6['Alpha'].str.contains(txt).any(): #https://www.statology.org/pandas-check-if-column-contains-string/
    if med_circ6['Alpha'].eq(txt).any(): #https://www.statology.org/pandas-check-if-column-contains-string/        
        return med_circ6.loc[med_circ6['Alpha'] == txt].iat[0,1]
        
txt1 = 'WHL'
txt2 = 'adsfasd'
print(medcirc062022(txt1), medcirc062022(txt2))

In [ ]:
# function to assign med scheme category based on Circular 12 of 2023 from the CMS
def medcirc122023(txt):
    #if med_circ12['Alpha'].str.contains(txt).any(): #https://www.statology.org/pandas-check-if-column-contains-string/
    if med_circ12['Alpha'].eq(txt).any(): #https://www.statology.org/pandas-check-if-column-contains-string/        
        return med_circ12.loc[med_circ12['Alpha'] == txt].iat[0,1]
        
txt1 = 'WHL'
txt2 = 'adsfasd'
print(medcirc122023(txt1), medcirc122023(txt2))

In [ ]:
# functions to extract CLN, FRN, NCD, ILB
def cln(txt):
    pattern = 'CLN|ASN|IVC|NN\d{3}'
    if re.search(pattern, str(txt).upper()):
        return 1
        
def frn(txt):
    pattern = 'STEP UP|STEP-UP|STEP_UP|STEPUP|NCD|FRN'
    if re.search(pattern, str(txt).upper()):
        return 1
        
def ilb(txt):
    pattern = 'INFLATION|CPI|ILB'
    if re.search(pattern, str(txt).upper()):
        return 1

txt1 = 'de Barge inflation asn'
txt2 = 'RSA R209 6.25% 310336'
txt  = 'PERFFEE'
print(frn(txt), type(frn(txt)), cln(txt), type(cln(txt)), ilb(txt), type(ilb(txt)))

In [ ]:
# function that indicates if a security is government guaranteed
def gvg(txt): #"vlookup" https://www.statology.org/pandas-check-if-column-contains-string/ exact string
    if (bond_data['Bond Code'].eq(txt)).any():
        if bond_data.loc[bond_data['Bond Code'] == txt].iat[0, 1] == 'GOVERNMENT GUARANTEE':    
            return 1
        
txt1 = 'ABF005'                        # = UNSUBORDINATED UNSECURED
txt2 = 'EL31'                          # = GOVERNMENT GUARANTEE
txt3 = 'This text is not in bond_data' #   test text
txt4 = ''                              #   test text
        
print(gvg(txt1), gvg(txt2), gvg(txt3), gvg(txt4))

In [ ]:
# function that indicates if bond security is BESA-listed
def besa(txt): #"vlookup" https://www.statology.org/pandas-check-if-column-contains-string/ exact string
    if (besa_data['Bond Code'].eq(txt)).any():
        return 'B'
        
txt1 = 'R186'                          # is in the BESA file
txt2 = 'COJ08'                         # is in the BESA file
txt3 = 'This text is not in bond_data' # not in the BESA file
txt4 = ''                              # not in the BESA file
print(besa(txt1), besa(txt2), besa(txt3), besa(txt4))

besa_date = datetime.strptime(max(ymd),'%Y%m%d')
days_diff = str((datetime.now() - datetime.strptime(max(ymd),'%Y%m%d')).days) # https://stackoverflow.com/questions/151199/how-to-calculate-number-of-days-between-two-given-dates
print('BESA file date is ' + besa_date.strftime('%d %B %Y') + ', ' + days_diff + ' days hence')

In [ ]:
# function that identifies a repo
def repo(txt):
    pattern = 'RPCO|RPMT|RPCA'
    if re.search(pattern, str(txt).upper()): 
        return txt[0:3]                # https://thispointer.com/python-how-to-get-first-n-characters-in-a-string/
    
txt1 = 'RPCA_R209_8.8_150623_160823'   # hit
txt2 = 'adsfasd'                       # miss
print(repo(txt1), repo(txt2))

In [ ]:
# function to get days remaining to maturity # https://www.geeksforgeeks.org/python-datetime-strptime-function/
def term(date_string):
    try:
        if len(date_string) == 0:
            return None
        else:
            return datetime.strptime(date_string, '%d%b%Y') - report_date
    
    # manage exceptions:
    except TypeError as te:
        return None
    except ValueError as ve:
        return None
    
txt1 = '31Dec2032'
txt2 = ''
print(term(txt1), type(term(txt1)), term(txt2), type(term(txt2)))

In [ ]:
example['Percentage of Market Value'].sum()

In [ ]:
fds = example['Entity Name'].dropna().unique()

In [ ]:
# function to assign an issuer
# https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d
def classify(row):
    t = issuer_did(row['i Issue Name'])                                           # temp so function only gets called once
    g = issuer_did(row['Primary Asset ID'])                                       # temp so function only gets called once

    if   row['Primary Asset ID'] in accr_list or row['Investment Type'] == 'FWD': # item is an accrual
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]        
    # https://www.geeksforgeeks.org/check-if-element-exists-in-list-in-python/
    
    elif row['Investment Type'] == 'SYTH':                                        # item is non-ZAR synthetic cash
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]
    
    #elif row['Investment Type'] == 'SYTH' and row['CCY'] != 'ZAR':               # item is non-ZAR synthetic cash
        #return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]
    
    #elif row['Investment Type'] == 'SYTH' and row['CCY'] == 'ZAR':               # item is ZAR synthetic cash
        #return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,2]
    
    elif row['Primary Asset ID'] in margin_list:                                  # item is a margin account (initial or variation)
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,2]
   
    elif row['Investment Type'] == 'OP' and 'ZAR' in row['Primary Asset ID']:     # item is a currency option
        return sttlmnt.loc[sttlmnt['Fund'] == row['Entity Name']].iat[0,1]    
    
    elif (clns['Code'].eq(row['Primary Asset ID'])).any():                        # cln is included in list
        return clns.loc[clns['Code'] == row['Primary Asset ID']].iat[0,1]   
    
    elif repo(row['Primary Asset ID']) == 'RPC':                                  # repo, bank legs RPCO and RPCA      
        return 'Absa Bank Ltd'  
    
    elif repo(row['Primary Asset ID']) == 'RPMT':                                 # repo, government bond leg                 
        return 'Republic of South Africa'      
    
    elif isinstance(g, str):                                                      # INSTRUMENT DESCRIPTION is a hit ...
        return g
    
    elif isinstance(t, str):                                                      # INSTRUMENT ID is a hit ...
        return t
    
    #return '-xxx-'                                                                
    else:                                                                         # signifies no issuer assigned
        return '-xxx'

In [ ]:
# (3) identify instrument attributes
start_time = time.time()

# Map an if statement in Python: https://stackoverflow.com/questions/29247718/map-an-if-statement-in-python
example['CLN']           = example['i Issue Name'    ].map(cln)
example['FRN']           = example['i Issue Name'    ].map(frn)
example['ILB']           = example['i Issue Name'    ].map(ilb)
example['Date']          = example['i Issue Name'    ].map(datex)         # security maturity date
example['MedCirc062022'] = example['Primary Asset ID'].map(medcirc062022)
example['MedCirc122023'] = example['Primary Asset ID'].map(medcirc122023)
example['GovGuar']       = example['Primary Asset ID'].map(gvg)
example['repo']          = example['Primary Asset ID'].map(repo)
example['margin']        = example['Primary Asset ID'].map(marg)
example['BESA']          = example['Primary Asset ID'].map(besa)

# https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# https://stackoverflow.com/questions/37840812/pandas-subtracting-two-date-columns-and-the-result-being-an-integer
example['Term']          = (pd.to_datetime(example['Date'], format = '%d%b%Y') - report_date).dt.days

print(timediff(start_time, time.time()))

In [ ]:
# (3) identify issuers
start_time = time.time()

example['Issuer'] = example.apply(lambda row: classify(row), axis=1)
# Map an if statement in Python: https://stackoverflow.com/questions/29247718/map-an-if-statement-in-python

print(str(len(issrgx['name'].dropna())) + ' regexes and ' + str(len(example)) + ' securities over ' + str(timediff(start_time, time.time())))

In [ ]:
# (4) identify securities with absent issuers
start_time = time.time()

# drop duplicate 'no issuer' rows https://www.interviewqs.com/ddi-code-snippets/drop-duplicate-rows-pandas
no_issuer     = example[(example['Issuer'] == '-xxx-') | (example['Issuer'].isnull())
                       ].drop_duplicates(subset='Primary Asset ID', keep = 'first')

# (5) identify securities with absent CLN issuers
# Pandas: How to Select Rows Based on Column Values https://www.statology.org/pandas-select-rows-based-on-column-values/
no_CLN_issuer = example[(example['CLN']   == 1)        & (example['Issuer'].isna())
                       ].drop_duplicates(subset='Primary Asset ID', keep = 'first')

print(timediff(start_time, time.time()))

In [ ]:
# (6) write the dataframe to review it as a workbook
start_time = time.time()

writer = pd.ExcelWriter(pthW + results_filename, engine   = 'xlsxwriter')                              #!pip install xlsxwriter
example.to_excel(       writer, index = False, sheet_name = 'results')                                 # assigned attributes
no_issuer.to_excel(     writer, index = False, sheet_name =  f'no issuers ({len(no_issuer)})')         # unassigned issuers
no_CLN_issuer.to_excel( writer, index = False, sheet_name =  f'no CLN issuers ({len(no_CLN_issuer)})') # unassigned CLNs
writer.save()
writer.close()
# stored here P:\Working Folders\Hilton\W\issueras_1_done.xlsx

print(timediff(start_time, time.time()))

In [ ]:
# (7) prettify the sheets using openpyxl
start_time = time.time()

# add filters to all columns https://stackoverflow.com/questions/51566349/openpyxl-how-to-add-filters-to-all-columns, and then
# freeze entire header row in openpyxl https://stackoverflow.com/questions/25588918/how-to-freeze-entire-header-row-in-openpyxl
# iterate over worksheets https://stackoverflow.com/questions/18495672/how-to-iterate-over-worksheets-in-workbook-openpyxl
# wrap text in sheet headers https://stackoverflow.com/questions/42215933/apply-wrap-text-to-all-cells-using-openpyxl
# fill colour cells https://stackoverflow.com/questions/30484220/fill-cells-with-colors-using-openpyxl
wb = px.load_workbook(pthW + results_filename)
for sheet in wb.worksheets:
    sheet.auto_filter.ref = sheet.dimensions
    sheet.freeze_panes = sheet['J2']
    for row in sheet.iter_rows(min_row=1, max_row=1):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True,vertical='top')
            cell.fill = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')

# add a hyperlink to the 'structures' sheet
#https://stackoverflow.com/questions/46162147/python-openpyxl-change-font-to-bold
#https://toricode.com/python-openpyxl-font-underline-for-excel-cells/
#https://stackoverflow.com/questions/8440284/setting-styles-in-openpyxl
#https://stackoverflow.com/questions/22986725/how-can-i-align-text-in-a-cell-to-the-top-with-openpyxl
sht                 = wb[f'no issuers ({len(no_issuer)})']      
sht['P1'].hyperlink = structures
sht['P1'].value     ='structures.xlsm'
sht['P1'].font      = Font(bold = True, underline = 'single', color = "0000EE")
sht['P1'].alignment = Alignment(horizontal = 'left', vertical = 'top')
            
wb.save(pthW + results_filename) # save the file to the W folder
wb.close()

print(timediff(start_time, time.time()))

In [ ]:
# (8) present how many securities were not allocated an issuer and ...
start_time = time.time()

k = ' ' if len(fds) == 1 else 's' # https://learnpython.com/blog/python-if-in-one-line/
print('Report date   : '        + str(report_date) + \
      '\n' + 'BESA file date: ' + max(ymd)         + \
      '\n' + str(len(fds))      + ' fund'          + k + '       : ' + ', '.join(np.sort(fds)),'\n')

print(  'Unallocated issuers:', str(len(no_issuer.Issuer)),
      '\nUnallocated CLNs   :', str(len(no_CLN_issuer.CLN)),'\n')

print(timediff(start_time, time.time())) # time for this cell to complete
print('Total time: ' + timediff(start_time_overlord, time.time())) # time for all the cells to complete

# dataframes: example, no_issuer, no_CLN_issuer
# input file  P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx
# structures  P:\Working Folders\Hilton\W\structures.xlsm
# merged      P:\Working Folders\Hilton\W\merged.xlsx
# output      P:\Working Folders\Hilton\W\output.xlsx
# results     P:\Working Folders\Hilton\W\issuers_1_done.xlsx

In [ ]:
if (len(no_issuer.Issuer) == 0) and (len(no_CLN_issuer.CLN) ==0):
    # https://stackoverflow.com/questions/20186344/importing-an-ipynb-file-from-another-ipynb-file   
    %run "C:/Users/hilton.netta/OneDrive - Prescient/py/gitrepo/issuers_2.ipynb"
    %run "C:/Users/hilton.netta/OneDrive - Prescient/py/gitrepo/issuers_3.ipynb"
else:
    excel.Workbooks.Open(r'P:\Working Folders\Hilton\W\issuers_1_done.xlsx')

In [ ]:
# dataframes    :example, no_issuer, no_CLN_issuer
# input file   P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx
# structures   P:\Working Folders\Hilton\W\structures.xlsm
# results      P:\Working Folders\Hilton\W\issuers_1_done.xlsx
# merged       P:\Working Folders\Hilton\W\merged.xlsx
# output       P:\Working Folders\Hilton\W\issuers_2_done.xlsx
# Reg 28 & 30  P:\Investment Operations\GRC\Compliance\Reg28 and Reg30 Reporting
# folder       P:\Working Folders\Hilton\W\Reg_Tests